# SBP semidiscretization of the linear advection equation

In [ ]:
include("setup.jl")

centroidal(a, b) = 2 * (a^2 + a*b + b^2) / (3 * (a + b))
armean(a, b) = (a + b) / 2
heronian(a, b) = (a + sqrt(a*b) + b) / 3
function logmean(a, b)
    if a ≈ b
        return (a + b) / 2
    else
        return (a - b) / log(a / b)
    end
end
geometric(a, b) = sqrt(a * b)
function harmonic(a, b)
    if a ≈ b
        return (a + b) / 2
    else
        return 2 * a * b / (a + b)
    end
end


function linear_advection!(du, u, param, t)
    @unpack D, Dop, numerical_flux = param
    
    du .= zero(eltype(du))
    
    for col in 1:size(D, 2)
        for idx in nzrange(D, col)
            row = rowvals(D)[idx]
            D_row_col = nonzeros(D)[idx]
            
            du[row] -= 2 * D_row_col * numerical_flux(u[row], u[col])
        end
    end
end

u_sol(t, x) = 2 + 1.9 * sinpi(x - t)
xmin(::typeof(u_sol)) = 0.0
xmax(::typeof(u_sol)) = 2.0

## Eigenvalues of SBP discretizations with increasing number of degrees of freedom

In [ ]:
function compute_eigenvalues(param, u0)
    J = ForwardDiff.jacobian((output,input) -> linear_advection!(output, input, param, 0.0),
                             similar(u0), u0)
    λ = eigvals(J)
end

function compute_eigenvalues_FD(numerical_flux, orders, n_nodes_list, u_sol)
    λs = Vector{Vector{ComplexF64}}()
    for (n_nodes, order) in Iterators.product(n_nodes_list, orders)
        Dop = periodic_derivative_operator(1, order, xmin(u_sol), xmax(u_sol), n_nodes+1)
        D = sparse(Dop)
        param = (; Dop, D, numerical_flux)
        x = grid(Dop)
        u0 = u_sol.(0.0, x)
        push!(λs, compute_eigenvalues(param, u0))
    end
    λs
end

function compute_eigenvalues_DG(numerical_flux, polydeg_list, n_elements_list, u_sol)
    λs = Vector{Vector{ComplexF64}}()
    for (n_elements, polydeg) in Iterators.product(n_elements_list, polydegs)
        Dop1 = legendre_derivative_operator(-1., 1., polydeg+1)
        mesh = UniformPeriodicMesh1D(xmin(u_sol), xmax(u_sol), n_elements)
        Dop = couple_discontinuosly(Dop1, mesh)
        D = sparse(Dop)
        param = (; Dop, D, numerical_flux)
        x = grid(Dop)
        u0 = u_sol.(0.0, x)
        push!(λs, compute_eigenvalues(param, u0))
    end
    λs
end

function compute_eigenvalues_CG(numerical_flux, polydeg_list, n_elements_list, u_sol)
    λs = Vector{Vector{ComplexF64}}()
    for (n_elements, polydeg) in Iterators.product(n_elements_list, polydegs)
        Dop1 = legendre_derivative_operator(-1., 1., polydeg+1)
        mesh = UniformPeriodicMesh1D(xmin(u_sol), xmax(u_sol), n_elements)
        Dop = couple_continuosly(Dop1, mesh)
        D = sparse(Dop)
        param = (; Dop, D, numerical_flux)
        x = grid(Dop)
        u0 = u_sol.(0.0, x)
        push!(λs, compute_eigenvalues(param, u0))
    end
    λs
end


function plot_eigenvalues(λs, labels, ncol)
#     maximum.(real, λs) |> display
#     maximum.(imag, λs) |> display
#     maximum.(abs, λs) |> display
    
    fig, ax = plt.subplots(1, 1)
    ax.set_prop_cycle(marker_cycler)
    for (idx, λ) in enumerate(λs)
        plt.plot(real.(λ), imag.(λ), label=labels[idx])
    end
    plt.xlabel(L"\mathrm{Re}(\lambda)")
    plt.ylabel(L"\mathrm{Im}(\lambda)")
    
#     plt.legend(loc="lower center", bbox_to_anchor=(0.45, 1.0), ncol=3, columnspacing=0)
    fig_legend = plt.figure()
    handles, labels = ax.get_legend_handles_labels()
    plt.figlegend(handles, labels, loc="center", ncol=ncol)
    
    fig, fig_legend
end

In [ ]:
orders = [2, 4]
n_nodes_list = [10, 100, 1000]
# n_nodes_list = [10, 11, 100, 101, 1000, 1001]
λs = compute_eigenvalues_FD(logmean, orders, n_nodes_list, u_sol)
labels = map(((n_nodes, order),) -> @sprintf("\$p = %d\$, \$%d\$ nodes", order, n_nodes), 
    Iterators.product(n_nodes_list, orders))
fig_FD, leg_FD = plot_eigenvalues(λs, labels, 2)
fig_FD.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_FD.pdf"), bbox_inches="tight")
leg_FD.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_FD_legend.pdf"), bbox_inches="tight")

polydegs = 3:4
n_elements_list = [3, 30, 300]
# n_elements_list = [3, 4, 30, 31, 300, 301]
λs = compute_eigenvalues_DG(logmean, polydegs, n_elements_list, u_sol)
labels = map(((n_elements, polydeg),) -> @sprintf("\$N = %d\$, \$%d\$ elements", polydeg, n_elements), 
    Iterators.product(n_elements_list, polydegs))
fig_DG, leg_DG = plot_eigenvalues(λs, labels, 2)
fig_DG.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_DG.pdf"), bbox_inches="tight")
leg_DG.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_DG_legend.pdf"), bbox_inches="tight")

polydegs = 3:4
n_elements_list = [3, 30, 300]
# n_elements_list = [3, 4, 30, 31, 300, 301]
λs = compute_eigenvalues_CG(logmean, polydegs, n_elements_list, u_sol)
labels = map(((n_elements, polydeg),) -> @sprintf("\$N = %d\$, \$%d\$ elements", polydeg, n_elements), 
    Iterators.product(n_elements_list, polydegs))
fig_CG, leg_CG = plot_eigenvalues(λs, labels, 2)
fig_CG.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_CG.pdf"), bbox_inches="tight")
leg_CG.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_CG_legend.pdf"), bbox_inches="tight")

## Eigenvalues of FD discretizations with different numerical fluxes

In [ ]:
function compute_eigenvalues_FD(numerical_flux_list, u_sol, labels)
    order = 2
    n_nodes = 1000
    λs = Vector{Vector{ComplexF64}}()
    Dop = periodic_derivative_operator(1, order, xmin(u_sol), xmax(u_sol), n_nodes+1)
    D = sparse(Dop)
    x = grid(Dop)
    u0 = u_sol.(x, 0.0)
    for (idx,numerical_flux) in enumerate(numerical_flux_list)
        param = (; Dop, D, numerical_flux)
        λ = compute_eigenvalues(param, u0)
        push!(λs, λ)
        fig, leg = plot_eigenvalues([λ], [labels[idx]], 1)
        fig.axes[1].set_xlim(-10, 10)
        fig.axes[1].set_ylim(-600, 600)
        fig.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_" * labels[idx] * ".pdf"), 
                    bbox_inches="tight")
    end
    λs
end

In [ ]:
numerical_flux_list = (centroidal, armean, heronian, logmean, geometric, harmonic)
labels = ["centroidal", "arithmetic", "Heronian", "logarithmic", "geometric", "harmonic"]
λs = compute_eigenvalues_FD(numerical_flux_list, u_sol, labels)
map(λ -> maximum(real, λ), λs)